In [ ]:
!pip install neo4j

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-google-genai

In [ ]:
!pip install langchain-neo4j

In [ ]:
!pip install langchain-experimental

In [ ]:
!pip install wikipedia

In [ ]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

In [ ]:
load_dotenv(dotenv_path='.env')
URI  = os.getenv('NEO4J_URL')
USER = os.getenv('NEO4J_USER')
PWD  = os.getenv('NEO4J_PWD')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [ ]:
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [ ]:
graph = Neo4jGraph(url=URI, username=USER, password=PWD, enhanced_schema=True)

In [ ]:
graph.refresh_schema()
graph.schema 

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=GOOGLE_API_KEY, temperature=0)

In [ ]:
chain  = GraphCypherQAChain.from_llm(
    llm, graph=graph, allow_dangerous_requests=True, verbose=True,return_intermediate_steps=True
)

In [ ]:
chain.invoke("give me list of iot devices")

In [ ]:
with open('iot.txt', 'r') as file:
    content = file.read()

In [ ]:
content

In [ ]:
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
from langchain_core.documents import Document

documents = [Document(page_content=content)]
graph_documents = await llm_transformer.aconvert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [ ]:
graph.add_graph_documents(graph_documents,include_source=True)

In [ ]:
graph.query("MATCH (n) RETURN count(n) AS node_count")

In [ ]:
from langchain.document_loaders import WikipediaLoader
text = WikipediaLoader(query="IOT Devices").load()

In [ ]:
text[0].metadata

In [ ]:
text = text[0:3]

In [ ]:
from langchain_core.documents import Document

graph_documents = await llm_transformer.aconvert_to_graph_documents(text)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{name:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    allow_dangerous_requests=True,
)

In [ ]:
chain.invoke({"query": "How many iot devices are there?"})